"[Список стран по ожидаемой продолжительности жизни](https://ru.wikipedia.org/wiki/Список_стран_по_ожидаемой_продолжительности_жизни)"<br />
"[Продолжительность жизни в субъектах Российской Федерации](https://ru.wikipedia.org/wiki/Продолжительность_жизни_в_субъектах_Российской_Федерации)"<br />
[fedstat.ru](https://www.fedstat.ru/indicator/31293) (ЕМИСС - Единая межведомственная информационно-статистическая система)<br />
[colors](http://mal-bioit.ru/programs/small/survey-web-colors.html)

In [1]:
import pandas as pd
import numpy as np
import math
import re

In [2]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 50

In [3]:
# load stats about longevity per year
df = pd.read_excel('data/Rosstat_by_year_and_region.xls', skiprows=2)

print(df.shape)
df.head(2)

(108, 35)


,Unnamed: 0,Unnamed: 1,Unnamed: 2,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Российская Федерация,Оба пола,все население,69.2,68.9,67.8,65.0,63.9,64.5,65.8,66.7,67.1,65.9,65.3,65.2,65.0,64.9,65.3,65.4,66.7,67.6,68.0,68.8,68.9,69.83,70.24,70.76,70.93,71.39,71.87,72.70,72.91,73.34,71.54,70.06
1,Центральный федеральный округ,Оба пола,все население,69.5,69.2,68.3,65.6,64.2,64.9,66.5,67.4,67.6,66.4,66.1,65.8,65.6,65.7,66.3,66.5,67.6,68.4,68.8,69.7,69.9,71.19,71.43,71.93,72.10,72.72,73.07,73.89,74.01,74.54,72.57,70.85


In [4]:
# Clean names of regions from surrounding spaces
df['Unnamed: 0'] = df['Unnamed: 0'].map(lambda st: st.strip())

# convert region column to index
df.set_index('Unnamed: 0', inplace=True)

# for brevityh of display remove name of index column.
df.index.name = ''

# sort index to speed up further transformations
df.sort_index(inplace=True)

# patch for Ingushetia for 1990-1992
# df.loc['Республика Ингушетия', '1990':'1992'] = df.loc['Чеченская и Ингушская Республики', '1990':'1992']

print(df.shape)
df.head(2).fillna(' ')

(108, 34)


,Unnamed: 1,Unnamed: 2,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Агинский Бурятский округ (Забайкальский край),Оба пола,все население,,,,62.4,59.9,62.7,62.3,63.9,63.5,62.8,61.9,61.3,62.2,,,,,,,,,,,,,,,,,,,
Алтайский край,Оба пола,все население,68.9,68.4,67.7,65.0,64.0,64.3,65.3,66.0,67.8,66.8,66.6,66.3,65.6,65.7,65.5,64.6,66.5,67.1,67.4,68.4,68.4,68.97,69.11,69.77,70.01,70.44,70.74,71.1,71.11,71.61,70.19,68.6


In [5]:
def filter_regions(df_regions: 'dataframe', errors='raise'):
    """
    The function takes dataframe with regions, remove some rows and rename others.
    """
    
    ls_dropping = [
        'Центральный федеральный округ',
        'Северо-Западный федеральный округ',
        'Южный федеральный округ (по 2009 год)',
        'Южный федеральный округ (с 2010 года)',
        'Южный федеральный округ (с 29.07.2016)',
        'Северо-Кавказский федеральный округ',
        'Приволжский федеральный округ',
        'Уральский федеральный округ',
        'Сибирский федеральный округ',
        'Дальневосточный федеральный округ',
        'Крымский федеральный округ',
        'Коми-Пермяцкий округ, входящий в состав Пермского края',
        'Таймырский (Долгано-Ненецкий) автономный округ (Красноярский край)',
        'Эвенкийский автономный округ (Красноярский край)',
        'Усть-Ордынский Бурятский округ',
        'Агинский Бурятский округ (Забайкальский край)',
        'Корякский округ, входящий в состав Камчатского края',
        'Чеченская и Ингушская Республики' 
    ]

    dict_renaming = {
        'Российская Федерация': 'Россия в среднем',
        'Город Москва столица Российской Федерации город федерального значения': 'г. Москва',
        'Город Санкт-Петербург город федерального значения': 'г. Санкт-Петербург',
        'Республика Адыгея (Адыгея)': 'Республика Адыгея',
        'Город федерального значения Севастополь': 'г. Севастополь',
        'Республика Татарстан (Татарстан)': 'Республика Татарстан',
        'Чувашская Республика - Чувашия': 'Чувашская Республика (Чувашия)',
        'Ханты-Мансийский автономный округ - Югра (Тюменская область)': 'Ханты-Мансийский АО - Югра (Тюменская область)',
        'Ямало-Ненецкий автономный округ (Тюменская область)': 'Ямало-Ненецкий АО (Тюменская область)',
        'Ненецкий автономный округ (Архангельская область)': 'Ненецкий АО (Архангельская область)',
        'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)': 'Тюменская область (кроме двух АО)',
        'Архангельская область (кроме Ненецкого автономного округа)': 'Архангельская область (кроме Ненецкого АО)',
        'Чукотский автономный округ': 'Чукотский АО'
    }
    
    return df_regions.drop(ls_dropping, errors=errors) \
                     .rename(index=dict_renaming, errors=errors) \
                     .sort_index()
    
df = filter_regions(df)
print(df.shape)
df.head(2)

(88, 34)


,Unnamed: 1,Unnamed: 2,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Алтайский край,Оба пола,все население,68.9,68.4,67.7,65.0,64.0,64.3,65.3,66.0,67.8,66.8,66.6,66.3,65.6,65.7,65.5,64.6,66.5,67.1,67.4,68.4,68.4,68.97,69.11,69.77,70.01,70.44,70.74,71.10,71.11,71.61,70.19,68.6
Амурская область,Оба пола,все население,67.8,67.4,65.8,62.8,62.1,63.0,62.9,63.6,64.5,63.1,62.2,61.3,61.1,60.9,60.3,60.2,62.1,63.8,63.4,64.2,64.4,64.82,65.12,66.38,67.00,67.27,68.28,69.06,69.11,68.66,67.38,66.3


In [6]:
# select only required columns
df = df[['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']]

print(df.shape)
df.head(2)

(88, 8)


,2014,2015,2016,2017,2018,2019,2020,2021
,,,,,,,,
Алтайский край,70.01,70.44,70.74,71.10,71.11,71.61,70.19,68.6
Амурская область,67.00,67.27,68.28,69.06,69.11,68.66,67.38,66.3


<br />
<br />
<br />

In [7]:
# load stats about longevity per gender ('genloc' is abbreviation for 'gender and location'
genloc = pd.read_excel('data/Rosstat_by_gender_and_location_2019.xls', skiprows=2)

print(genloc.shape)
genloc.head(10)

(864, 4)


,Unnamed: 0,Unnamed: 1,Unnamed: 2,2019
0,Российская Федерация,Женщины,все население,78.17
1,Российская Федерация,Женщины,городское население,78.41
2,Российская Федерация,Женщины,сельское население,77.39
3,Российская Федерация,Мужчины,все население,68.24
4,Российская Федерация,Мужчины,городское население,68.56
5,Российская Федерация,Мужчины,сельское население,67.36
6,Российская Федерация,Оба пола,все население,73.34
7,Российская Федерация,Оба пола,городское население,73.72
8,Российская Федерация,Оба пола,сельское население,72.21
9,Центральный федеральный округ,Женщины,все население,79.06


In [8]:
# rename columns
genloc.rename(columns={'Unnamed: 0': 'region', 'Unnamed: 1': 'gender', 'Unnamed: 2': 'location'}, inplace=True)

# remove surrounding spaces in the column with names of regions
genloc['region'] = genloc['region'].map(lambda st: st.strip())

genloc.head(10)

,region,gender,location,2019
0,Российская Федерация,Женщины,все население,78.17
1,Российская Федерация,Женщины,городское население,78.41
2,Российская Федерация,Женщины,сельское население,77.39
3,Российская Федерация,Мужчины,все население,68.24
4,Российская Федерация,Мужчины,городское население,68.56
5,Российская Федерация,Мужчины,сельское население,67.36
6,Российская Федерация,Оба пола,все население,73.34
7,Российская Федерация,Оба пола,городское население,73.72
8,Российская Федерация,Оба пола,сельское население,72.21
9,Центральный федеральный округ,Женщины,все население,79.06


In [9]:
# make dataframe multi-index
genloc = genloc.sort_values(by=['region', 'gender', 'location']) \
               .set_index(['region', 'gender', 'location'])

print(genloc.shape)
genloc.head(10)

(864, 1)


2019
region           gender   location                  
Алтайский край   Женщины  все население        76.77
                          городское население  77.33
                          сельское население   75.77
                 Мужчины  все население        66.31
                          городское население  66.65
                          сельское население   65.62
                 Оба пола все население        71.61
                          городское население  72.22
                          сельское население   70.57
Амурская область Женщины  все население        73.91

In [10]:
# filter records: remove some ot them and rename others
genloc = filter_regions(genloc, errors='ignore')

print(genloc.shape)
genloc.head(10)

(792, 1)


2019
region           gender   location                  
Алтайский край   Женщины  все население        76.77
                          городское население  77.33
                          сельское население   75.77
                 Мужчины  все население        66.31
                          городское население  66.65
                          сельское население   65.62
                 Оба пола все население        71.61
                          городское население  72.22
                          сельское население   70.57
Амурская область Женщины  все население        73.91

In [11]:
# reshape dataframe, transforming the last two levels of multi-index to columns
genloc = genloc.unstack().unstack()
genloc.head(2)

2019                                               \
location         все население                  городское население           
gender                 Женщины Мужчины Оба пола             Женщины Мужчины   
region                                                                        
Алтайский край           76.77   66.31    71.61               77.33   66.65   
Амурская область         73.91   63.52    68.66               74.48   63.73   

                                                               
location                  сельское население                   
gender           Оба пола            Женщины Мужчины Оба пола  
region                                                         
Алтайский край      72.22              75.77   65.62    70.57  
Амурская область    69.19              72.40   63.05    67.43

In [12]:
# simplify heading
heading = genloc.columns
genloc.columns = map(lambda tp: '-'.join(tp[1:]), heading)

genloc.head(2)

,все население-Женщины,все население-Мужчины,все население-Оба пола,городское население-Женщины,городское население-Мужчины,городское население-Оба пола,сельское население-Женщины,сельское население-Мужчины,сельское население-Оба пола
region,,,,,,,,,
Алтайский край,76.77,66.31,71.61,77.33,66.65,72.22,75.77,65.62,70.57
Амурская область,73.91,63.52,68.66,74.48,63.73,69.19,72.40,63.05,67.43


In [13]:
# choose only required to us columns and place them in required order
genloc = genloc[[
    'все население-Оба пола',
    'все население-Мужчины',
    'все население-Женщины',
    'городское население-Оба пола',
    'сельское население-Оба пола'    
]].rename(columns={
    'все население-Оба пола': 'overall',
    'все население-Мужчины': 'males',
    'все население-Женщины': 'females',
    'городское население-Оба пола': 'urban',
    'сельское население-Оба пола': 'rural'
})


# if in some column there's 0, than change it to NaN (it's actual for Saint Petersburg)
genloc.replace(0, np.nan, inplace=True)

genloc.head(2)

,overall,males,females,urban,rural
region,,,,,
Алтайский край,71.61,66.31,76.77,72.22,70.57
Амурская область,68.66,63.52,73.91,69.19,67.43


<br />

In [14]:
# before joining two dataframes make sure correctness of transformations
assert len(df) == len(genloc), "Problem wiht length of dataframes"
assert all(df.index == genloc.index), "Problem with indexes of dataframes"
assert all(df['2019'] == genloc['overall']), "content of the column '2019' is not identical to columns 'all_population'"

In [15]:
# concatinate info from two database
df = pd.concat([genloc, df], axis='columns')
df.head(2)

,overall,males,females,urban,rural,2014,2015,2016,2017,2018,2019,2020,2021
Алтайский край,71.61,66.31,76.77,72.22,70.57,70.01,70.44,70.74,71.10,71.11,71.61,70.19,68.6
Амурская область,68.66,63.52,73.91,69.19,67.43,67.00,67.27,68.28,69.06,69.11,68.66,67.38,66.3


In [16]:
# calculate difference between some columns and insert result as new columns in dataframe
df.insert(loc=3, column='females_vs_males', value=df['females']-df['males'])
df.insert(loc=6, column='urban_vs_rural', value=df['urban']-df['rural'])
df.insert(loc=8, column='2014→2015', value=df['2015']-df['2014'])
df.insert(loc=10, column='2015→2016', value=df['2016']-df['2015'])
df.insert(loc=12, column='2016→2017', value=df['2017']-df['2016'])
df.insert(loc=14, column='2017→2018', value=df['2018']-df['2017'])
df.insert(loc=16, column='2018→2019', value=df['2019']-df['2018'])
df.insert(loc=18, column='2019→2020', value=df['2020']-df['2019'])
df.insert(loc=20, column='2020→2021', value=df['2021']-df['2020'])
df.head(2)

,overall,males,females,females_vs_males,urban,rural,urban_vs_rural,2014,2014→2015,2015,2015→2016,2016,2016→2017,2017,2017→2018,2018,2018→2019,2019,2019→2020,2020,2020→2021,2021
Алтайский край,71.61,66.31,76.77,10.46,72.22,70.57,1.65,70.01,0.43,70.44,0.30,70.74,0.36,71.10,0.01,71.11,0.50,71.61,-1.42,70.19,-1.59,68.6
Амурская область,68.66,63.52,73.91,10.39,69.19,67.43,1.76,67.00,0.27,67.27,1.01,68.28,0.78,69.06,0.05,69.11,-0.45,68.66,-1.28,67.38,-1.08,66.3


In [17]:
# sort table by 'all_population'
df.sort_values(by=['overall', '2019', '2018'], ascending=False, inplace=True)

In [18]:
# move some records to the top of dataframe (by default, column 'Россия в среднем')
# change order of some regions
indexes = df.index.to_list()
ls_change_order = ['Россия в среднем']    # 'г. Москва', 'г. Санкт-Петербург', 'г. Севастополь'
indexes = list(el for el in indexes if el not in ls_change_order)
indexes = ls_change_order + indexes
df = df.reindex(indexes)

In [19]:
ls_dropping = [
    'Архангельская область',
    'Тюменская область'
]

In [20]:
df_subjects = df.drop(index=ls_dropping)
print(df_subjects.shape)
df_subjects.fillna(' ')

(86, 22)


,overall,males,females,females_vs_males,urban,rural,urban_vs_rural,2014,2014→2015,2015,2015→2016,2016,2016→2017,2017,2017→2018,2018,2018→2019,2019,2019→2020,2020,2020→2021,2021
Россия в среднем,73.34,68.24,78.17,9.93,73.72,72.21,1.51,70.93,0.46,71.39,0.48,71.87,0.83,72.70,0.21,72.91,0.43,73.34,-1.80,71.54,-1.48,70.06
Республика Ингушетия,83.40,80.00,86.27,6.27,83.40,83.39,0.01,79.42,0.63,80.05,0.77,80.82,0.77,81.59,0.82,82.41,0.99,83.40,-1.92,81.48,-0.96,80.52
Республика Дагестан,79.10,76.62,81.44,4.82,79.85,78.35,1.5,75.83,0.56,76.39,0.84,77.23,0.56,77.79,0.90,78.69,0.41,79.10,-2.67,76.43,0.16,76.59
г. Москва,78.36,74.83,81.71,6.88,78.39,75.79,2.6,76.70,0.07,76.77,0.31,77.08,0.79,77.87,-0.03,77.84,0.52,78.36,-2.16,76.20,-1.65,74.55
Кабардино-Балкарская Республика,76.46,72.62,79.94,7.32,76.71,76.19,0.52,74.16,0.45,74.61,0.51,75.12,0.69,75.81,0.47,76.28,0.18,76.46,-2.09,74.37,-0.60,73.77
г. Санкт-Петербург,76.31,71.99,79.99,8.00,76.31,,,74.57,-0.15,74.42,0.48,74.90,0.55,75.45,0.48,75.93,0.38,76.31,-2.32,73.99,-1.48,72.51
Карачаево-Черкесская Республика,76.21,71.68,80.40,8.72,76.53,75.9,0.63,73.91,0.53,74.44,0.28,74.72,1.22,75.94,0.15,76.09,0.12,76.21,-1.39,74.82,-1.35,73.47
Чеченская Республика,75.88,73.61,77.97,4.36,73.73,77.07,-3.34,73.06,0.39,73.45,0.75,74.20,0.64,74.84,0.59,75.43,0.45,75.88,-3.77,72.11,0.89,73.00
Республика Северная Осетия-Алания,75.75,70.52,80.57,10.05,75.87,75.56,0.31,73.82,0.38,74.20,0.85,75.05,0.46,75.51,0.17,75.68,0.07,75.75,-1.67,74.08,-1.61,72.47
Ханты-Мансийский АО - Югра (Тюменская область),75.04,70.60,79.28,8.68,75.24,72.59,2.65,72.27,0.31,72.58,0.92,73.50,0.37,73.87,0.41,74.28,0.76,75.04,-2.17,72.87,-0.86,72.01


In [21]:
df_subjects.head(1)

,overall,males,females,females_vs_males,urban,rural,urban_vs_rural,2014,2014→2015,2015,2015→2016,2016,2016→2017,2017,2017→2018,2018,2018→2019,2019,2019→2020,2020,2020→2021,2021
Россия в среднем,73.34,68.24,78.17,9.93,73.72,72.21,1.51,70.93,0.46,71.39,0.48,71.87,0.83,72.7,0.21,72.91,0.43,73.34,-1.8,71.54,-1.48,70.06


In [22]:
dd_replacement_ru = {
    'Россия в среднем': 'Россия в среднем',
    'Республика Ингушетия': '[[Ингушетия]]',
    'Республика Дагестан': '[[Дагестан]]',
    'г. Москва': '[[Москва]]',
    'Кабардино-Балкарская Республика': '[[Кабардино-Балкария]]',
    'Карачаево-Черкесская Республика': '[[Карачаево-Черкесия]]',
    'Чеченская Республика': '[[Чечня]]',
    'г. Санкт-Петербург': '[[Санкт-Петербург]]',
    'Республика Северная Осетия-Алания': '[[Северная Осетия]]',
    'г. Севастополь': '[[Севастополь]]',
    'Ханты-Мансийский АО - Югра (Тюменская область)': '[[Ханты-Мансийский автономный округ — Югра|Ханты-Мансийский АО]]<br>(Тюменская область)',
    'Ямало-Ненецкий АО (Тюменская область)': '[[Ямало-Ненецкий автономный округ|Ямало-Ненецкий АО]]<br>(Тюменская область)',
    'Ставропольский край': '[[Ставропольский край]]',
    'Республика Калмыкия': '[[Калмыкия]]',
    'Республика Татарстан': '[[Татарстан]]',
    'Республика Адыгея': '[[Адыгея]]',
    'Калининградская область': '[[Калининградская область]]',
    'Белгородская область': '[[Белгородская область]]',
    'Краснодарский край': '[[Краснодарский край]]',
    'Московская область': '[[Московская область]]',
    'Республика Мордовия': '[[Мордовия]]',
    'Ленинградская область': '[[Ленинградская область]]',
    'Тюменская область (кроме двух АО)': '[[Тюменская область]]<br>(кроме двух АО)',
    'Чувашская Республика (Чувашия)': '[[Чувашия]]',
    'Удмуртская Республика': '[[Удмуртия]]',
    'Республика Саха (Якутия)': '[[Якутия|Саха (Якутия)]]',
    'Пензенская область': '[[Пензенская область]]',
    'Волгоградская область': '[[Волгоградская область]]',
    'Астраханская область': '[[Астраханская область]]',
    'Тамбовская область': '[[Тамбовская область]]',
    'Ростовская область': '[[Ростовская область]]',
    'Кировская область': '[[Кировская область]]',
    'Республика Крым': '[[Республика Крым]]',
    'Томская область': '[[Томская область]]',
    'Архангельская область (кроме Ненецкого АО)': '[[Архангельская область]]<br>(кроме АО)',
    'Воронежская область': '[[Воронежская область]]',
    'Республика Башкортостан': '[[Башкортостан]]',
    'Республика Марий Эл': '[[Марий Эл]]',
    'Ненецкий АО (Архангельская область)': '[[Ненецкий автономный округ|Ненецкий АО]]<br>(Архангельская область)',
    'Самарская область': '[[Самарская область]]',
    'Новосибирская область': '[[Новосибирская область]]',
    'Калужская область': '[[Калужская область]]',
    'Челябинская область': '[[Челябинская область]]',
    'Саратовская область': '[[Саратовская область]]',
    'Вологодская область': '[[Вологодская область]]',
    'Ярославская область': '[[Ярославская область]]',
    'Ульяновская область': '[[Ульяновская область]]',
    'Ивановская область': '[[Ивановская область]]',
    'Омская область': '[[Омская область]]',
    'Тульская область': '[[Тульская область]]',
    'Орловская область': '[[Орловская область]]',
    'Нижегородская область': '[[Нижегородская область]]',
    'Республика Бурятия': '[[Бурятия]]',
    'Свердловская область': '[[Свердловская область]]',
    'Костромская область': '[[Костромская область]]',
    'Брянская область': '[[Брянская область]]',
    'Рязанская область': '[[Рязанская область]]',
    'Приморский край': '[[Приморский край]]',
    'Алтайский край': '[[Алтайский край]]',
    'Липецкая область': '[[Липецкая область]]',
    'Курская область': '[[Курская область]]',
    'Пермский край': '[[Пермский край]]',
    'Республика Хакасия': '[[Хакасия]]',
    'Сахалинская область': '[[Сахалинская область]]',
    'Красноярский край': '[[Красноярский край]]',
    'Республика Коми': '[[Республика Коми|Коми]]',
    'Курганская область': '[[Курганская область]]',
    'Мурманская область': '[[Мурманская область]]',
    'Оренбургская область': '[[Оренбургская область]]',
    'Владимирская область': '[[Владимирская область]]',
    'Камчатский край': '[[Камчатский край]]',
    'Смоленская область': '[[Смоленская область]]',
    'Тверская область': '[[Тверская область]]',
    'Республика Алтай': '[[Республика Алтай|Алтай]]',
    'Хабаровский край': '[[Хабаровский край]]',
    'Псковская область': '[[Псковская область]]',
    'Новгородская область': '[[Новгородская область]]',
    'Кемеровская область - Кузбасс': '[[Кемеровская область|Кузбасс]]',
    'Магаданская область': '[[Магаданская область]]',
    'Республика Карелия': '[[Карелия]]',
    'Республика Тыва': '[[Тыва]]',
    'Забайкальский край': '[[Забайкальский край]]',
    'Иркутская область': '[[Иркутская область]]',
    'Амурская область': '[[Амурская область]]',
    'Еврейская автономная область': '[[Еврейская автономная область|Еврейская авт. область]]',
    'Чукотский АО': '[[Чукотский автономный округ|Чукотка]]',
    'Архангельская область': '[[Архангельская область]]',
    'Тюменская область': '[[Тюменская область]]'
}

### version with long names
# dd_replacement_ru = {
#     'Россия в среднем': 'Россия в среднем',
#     'Республика Ингушетия': '[[Ингушетия|Республика Ингушетия]]',
#     'Республика Дагестан': '[[Дагестан|Республика Дагестан]]',
#     'г. Москва': '[[Москва|г. Москва]]',
#     'Кабардино-Балкарская Республика': '[[Кабардино-Балкария|Кабардино-Балкарская Республика]]',
#     'Карачаево-Черкесская Республика': '[[Карачаево-Черкесия|Карачаево-Черкесская Республика]]',
#     'Чеченская Республика': '[[Чечня|Чеченская Республика]]',
#     'г. Санкт-Петербург': '[[Санкт-Петербург|г. Санкт-Петербург]]',
#     'Республика Северная Осетия-Алания': '[[Северная Осетия|Республика Северная Осетия]]',
#     'г. Севастополь': '[[Севастополь|г. Севастополь]]',
#     'Ханты-Мансийский АО - Югра (Тюменская область)': '[[Ханты-Мансийский автономный округ — Югра|Ханты-Мансийский АО — Югра]]<br>(Тюменская область)',
#     'Ямало-Ненецкий АО (Тюменская область)': '[[Ямало-Ненецкий автономный округ|Ямало-Ненецкий АО]]<br>(Тюменская область)',
#     'Ставропольский край': '[[Ставропольский край]]',
#     'Республика Калмыкия': '[[Калмыкия|Республика Калмыкия]]',
#     'Республика Татарстан': '[[Татарстан|Республика Татарстан]]',
#     'Республика Адыгея': '[[Адыгея|Республика Адыгея]]',
#     'Калининградская область': '[[Калининградская область]]',
#     'Белгородская область': '[[Белгородская область]]',
#     'Краснодарский край': '[[Краснодарский край]]',
#     'Московская область': '[[Московская область]]',
#     'Республика Мордовия': '[[Мордовия|Республика Мордовия]]',
#     'Ленинградская область': '[[Ленинградская область]]',
#     'Тюменская область (кроме двух АО)': '[[Тюменская область]]<br>(кроме двух АО)',
#     'Чувашская Республика (Чувашия)': '[[Чувашия|Чувашская Республика (Чувашия)]]',
#     'Удмуртская Республика': '[[Удмуртия|Удмуртская Республика]]',
#     'Республика Саха (Якутия)': '[[Якутия|Республика Саха (Якутия)]]',
#     'Пензенская область': '[[Пензенская область]]',
#     'Волгоградская область': '[[Волгоградская область]]',
#     'Астраханская область': '[[Астраханская область]]',
#     'Тамбовская область': '[[Тамбовская область]]',
#     'Ростовская область': '[[Ростовская область]]',
#     'Кировская область': '[[Кировская область]]',
#     'Республика Крым': '[[Республика Крым]]',
#     'Томская область': '[[Томская область]]',
#     'Архангельская область (кроме Ненецкого АО)': '[[Архангельская область]]<br>(кроме Ненецкого АО)',
#     'Воронежская область': '[[Воронежская область]]',
#     'Республика Башкортостан': '[[Башкортостан|Республика Башкортостан]]',
#     'Республика Марий Эл': '[[Марий Эл|Республика Марий Эл]]',
#     'Ненецкий АО (Архангельская область)': '[[Ненецкий автономный округ|Ненецкий АО]]<br>(Архангельская область)',
#     'Самарская область': '[[Самарская область]]',
#     'Новосибирская область': '[[Новосибирская область]]',
#     'Калужская область': '[[Калужская область]]',
#     'Челябинская область': '[[Челябинская область]]',
#     'Саратовская область': '[[Саратовская область]]',
#     'Вологодская область': '[[Вологодская область]]',
#     'Ярославская область': '[[Ярославская область]]',
#     'Ульяновская область': '[[Ульяновская область]]',
#     'Ивановская область': '[[Ивановская область]]',
#     'Омская область': '[[Омская область]]',
#     'Тульская область': '[[Тульская область]]',
#     'Орловская область': '[[Орловская область]]',
#     'Нижегородская область': '[[Нижегородская область]]',
#     'Республика Бурятия': '[[Бурятия|Республика Бурятия]]',
#     'Свердловская область': '[[Свердловская область]]',
#     'Костромская область': '[[Костромская область]]',
#     'Брянская область': '[[Брянская область]]',
#     'Рязанская область': '[[Рязанская область]]',
#     'Приморский край': '[[Приморский край]]',
#     'Алтайский край': '[[Алтайский край]]',
#     'Липецкая область': '[[Липецкая область]]',
#     'Курская область': '[[Курская область]]',
#     'Пермский край': '[[Пермский край]]',
#     'Республика Хакасия': '[[Хакасия|Республика Хакасия]]',
#     'Сахалинская область': '[[Сахалинская область]]',
#     'Красноярский край': '[[Красноярский край]]',
#     'Республика Коми': '[[Республика Коми]]',
#     'Курганская область': '[[Курганская область]]',
#     'Мурманская область': '[[Мурманская область]]',
#     'Оренбургская область': '[[Оренбургская область]]',
#     'Владимирская область': '[[Владимирская область]]',
#     'Камчатский край': '[[Камчатский край]]',
#     'Смоленская область': '[[Смоленская область]]',
#     'Тверская область': '[[Тверская область]]',
#     'Республика Алтай': '[[Республика Алтай]]',
#     'Хабаровский край': '[[Хабаровский край]]',
#     'Псковская область': '[[Псковская область]]',
#     'Новгородская область': '[[Новгородская область]]',
#     'Кемеровская область - Кузбасс': '[[Кемеровская область|Кемеровская область — Кузбасс]]',
#     'Магаданская область': '[[Магаданская область]]',
#     'Республика Карелия': '[[Карелия|Республика Карелия]]',
#     'Республика Тыва': '[[Тыва|Республика Тыва]]',
#     'Забайкальский край': '[[Забайкальский край]]',
#     'Иркутская область': '[[Иркутская область]]',
#     'Амурская область': '[[Амурская область]]',
#     'Еврейская автономная область': '[[Еврейская автономная область]]',
#     'Чукотский АО': '[[Чукотский автономный округ|Чукотский АО]]',
#     'Архангельская область': '[[Архангельская область]]',
#     'Тюменская область': '[[Тюменская область]]'
# }

In [23]:
# # create code for placing info in Wikipedia
# def create_table(df, file_header, file_footer, dd_replacement):

#     def if_value(x, prec=2):
#         return '-' if math.isnan(x) else f"{x:0.{prec}f}" #"{x:0.{prec}f}".format(x, prec)
    
#     with open('design/' + file_header, mode='r', encoding="utf-8") as fh:
#         table_header = fh.read()
    
#     with open('design/' + file_footer, mode='r', encoding="utf-8") as fh:
#         table_footer = fh.read()

#     st = table_header
#     for i in range(len(df)):
#         ser = df.iloc[i]
#         if ser.name == 'Россия в среднем':
#             st += '\n' + '|-class=static-row-header\n' + \
#                   f'|align=left|\'\'\'{dd_replacement[ser.name]}\'\'\' ' + \
#                   f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["overall"], 2)}\'\'\' ' + \
#                   f'|| style="background:#eaf3ff;"|\'\'\'{if_value(ser["males"], 2)}\'\'\' ' + \
#                   f'|| style="background:#fee7f6;"|\'\'\'{if_value(ser["females"], 2)}\'\'\' ' + \
#                   f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["females_vs_males"], 2)}\'\'\' ' + \
#                   f'|| style="border-left-width:2px;"|\'\'\'{if_value(ser["urban"], 2)}\'\'\' ' + \
#                   f'|| \'\'\'{if_value(ser["rural"], 2)}\'\'\' ' + \
#                   f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["urban_vs_rural"], 2)}\'\'\' ' + \
#                   f'|| style="border-left-width:2px;"|\'\'\'{if_value(ser["2014"], 2)}\'\'\' ' + \
#                   f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["2014→2015"], 2)}\'\'\' ' + \
#                   f'|| \'\'\'{if_value(ser["2015"], 2)}\'\'\' ' + \
#                   f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["2015→2016"], 2)}\'\'\' ' + \
#                   f'|| \'\'\'{if_value(ser["2016"], 2)}\'\'\' ' + \
#                   f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["2016→2017"], 2)}\'\'\' ' + \
#                   f'|| \'\'\'{if_value(ser["2017"], 2)}\'\'\' ' + \
#                   f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["2017→2018"], 2)}\'\'\' ' + \
#                   f'|| \'\'\'{if_value(ser["2018"], 2)}\'\'\' ' + \
#                   f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["2018→2019"], 2)}\'\'\' ' + \
#                   f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["2019"], 2)}\'\'\' ' + \
#                   f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["2019→2020"], 2)}\'\'\' ' + \
#                   f'|| \'\'\'{if_value(ser["2020"], 2)}\'\'\' ' + \
#                   f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["2020→2021"], 2)}\'\'\' ' + \
#                   f'|| \'\'\'{if_value(ser["2021"], 2)}\'\'\''
#         else:        
#             st += '\n' + '|-\n' + \
#                   f'|align=left|{dd_replacement[ser.name]} ' + \
#                   f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["overall"], 2)}\'\'\' ' + \
#                   f'|| style="background:#eaf3ff;"|{if_value(ser["males"], 2)} ' + \
#                   f'|| style="background:#fee7f6;"|{if_value(ser["females"], 2)} ' + \
#                   f'|| style="background:#fff8dc;"|{if_value(ser["females_vs_males"], 2)} ' + \
#                   f'|| style="border-left-width:2px;"|{if_value(ser["urban"], 2)} ' + \
#                   f'|| {if_value(ser["rural"], 2)} ' + \
#                   f'|| style="background:#fff8dc;"|{if_value(ser["urban_vs_rural"], 2)} ' + \
#                   f'|| style="border-left-width:2px;"|{if_value(ser["2014"], 2)} ' + \
#                   f'|| style="background:#fff8dc;"|{if_value(ser["2014→2015"], 2)} ' + \
#                   f'|| {if_value(ser["2015"], 2)} ' + \
#                   f'|| style="background:#fff8dc;"|{if_value(ser["2015→2016"], 2)} ' + \
#                   f'|| {if_value(ser["2016"], 2)} ' + \
#                   f'|| style="background:#fff8dc;"|{if_value(ser["2016→2017"], 2)} ' + \
#                   f'|| {if_value(ser["2017"], 2)} ' + \
#                   f'|| style="background:#fff8dc;"|{if_value(ser["2017→2018"], 2)} ' + \
#                   f'|| {if_value(ser["2018"], 2)} ' + \
#                   f'|| style="background:#fff8dc;"|{if_value(ser["2018→2019"], 2)} ' + \
#                   f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["2019"], 2)}\'\'\' ' + \
#                   f'|| style="background:#fff8dc;"|{if_value(ser["2019→2020"], 2)} ' + \
#                   f'|| {if_value(ser["2020"], 2)} ' + \
#                   f'|| style="background:#fff8dc;"|{if_value(ser["2020→2021"], 2)} ' + \
#                   f'|| {if_value(ser["2021"], 2)}'
#     st += '\n' + table_footer
#     return st

# table_code = create_table(df_subjects, file_header='rosstat_header_2019_ru.txt', file_footer='rosstat_footer_2019_ru.txt', dd_replacement=dd_replacement_ru)

# # write the code to file
# with open('output/Table code for Russian regions -2019 -ru.txt', 'w', encoding="utf-8") as fh:
#     fh.write(table_code)

<br />
<br />
<br />

In [24]:
# create code for placing info in Wikipedia
def create_table_advanced(df, file_header, file_footer, dd_replacement, lang='ru'):

    def if_value(x, prec=2):
        return '—' if math.isnan(x) else \
               f"{x:0.{prec}f}"  if x>=0 else \
               f"−{-x:0.{prec}f}"                #"{x:0.{prec}f}".format(x, prec)
    
    def chval(x, prec=2):  # change_value
        return '—' if math.isnan(x) else \
               f'style="background:#fffae0;color:darkgreen;"|{x:0.{prec}f}' if x>0 else \
               f'style="background:#fffae0;color:crimson;"|−{-x:0.{prec}f}' if x<0 else \
               f'style="background:#fffae0;color:darkgray;"|{x:0.{prec}f}'
    
    def chval_bold(x, prec=2):  # change_value
        return '—' if math.isnan(x) else \
               f'style="background:#fffae0;color:darkgreen;"|\'\'\'{x:0.{prec}f}\'\'\'' if x>0 else \
               f'style="background:#fffae0;color:crimson;"|\'\'\'−{-x:0.{prec}f}\'\'\'' if x<0 else \
               f'style="background:#fffae0;color:darkgray;"|\'\'\'{x:0.{prec}f}\'\'\''
    
    with open('design/' + file_header, mode='r', encoding="utf-8") as fh:
        table_header = fh.read()
    
    with open('design/' + file_footer, mode='r', encoding="utf-8") as fh:
        table_footer = fh.read()

    st = ''
    for i in range(len(df)):
        ser = df.iloc[i]
        if ser.name == 'Россия в среднем':
            st += '\n' + '|-class=static-row-header\n' + \
                  f'|style="text-align:left;"|\'\'\'{dd_replacement[ser.name]}\'\'\' ' + \
                  f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["overall"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center;background:#eaf3ff;"|\'\'\'{if_value(ser["males"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center;background:#fee7f6;"|\'\'\'{if_value(ser["females"], 2)}\'\'\' ' + \
                  f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["females_vs_males"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center;border-left-width:2px;"|\'\'\'{if_value(ser["urban"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center;"|\'\'\'{if_value(ser["rural"], 2)}\'\'\' ' + \
                  f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["urban_vs_rural"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center;border-left-width:2px;"|\'\'\'{if_value(ser["2014"], 2)}\'\'\' ' + \
                  f'|| {chval_bold(ser["2014→2015"], 2)} ' + \
                  f'|| style="text-align:center;"|\'\'\'{if_value(ser["2015"], 2)}\'\'\' ' + \
                  f'|| {chval_bold(ser["2015→2016"], 2)} ' + \
                  f'|| style="text-align:center;"|\'\'\'{if_value(ser["2016"], 2)}\'\'\' ' + \
                  f'|| {chval_bold(ser["2016→2017"], 2)} ' + \
                  f'|| style="text-align:center;"|\'\'\'{if_value(ser["2017"], 2)}\'\'\' ' + \
                  f'|| {chval_bold(ser["2017→2018"], 2)} ' + \
                  f'|| style="text-align:center;"|\'\'\'{if_value(ser["2018"], 2)}\'\'\' ' + \
                  f'|| {chval_bold(ser["2018→2019"], 2)} ' + \
                  f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["2019"], 2)}\'\'\' ' + \
                  f'|| {chval_bold(ser["2019→2020"], 2)} ' + \
                  f'|| style="text-align:center;"|\'\'\'{if_value(ser["2020"], 2)}\'\'\' ' + \
                  f'|| {chval_bold(ser["2020→2021"], 2)} ' + \
                  f'|| style="text-align:center;"|\'\'\'{if_value(ser["2021"], 2)}\'\'\''
        else:        
            st += '\n' + '|-\n' + \
                  f'|style="text-align:left;"|{dd_replacement[ser.name]} ' + \
                  f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["overall"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center;background:#eaf3ff;"|{if_value(ser["males"], 2)} ' + \
                  f'|| style="text-align:center;background:#fee7f6;"|{if_value(ser["females"], 2)} ' + \
                  f'|| style="background:#fff8dc;"|{if_value(ser["females_vs_males"], 2)} ' + \
                  f'|| style="text-align:center;border-left-width:2px;"|{if_value(ser["urban"], 2)} ' + \
                  f'|| style="text-align:center;"|{if_value(ser["rural"], 2)} ' + \
                  f'|| style="background:#fff8dc;"|{if_value(ser["urban_vs_rural"], 2)} ' + \
                  f'|| style="text-align:center;border-left-width:2px;"|{if_value(ser["2014"], 2)} ' + \
                  f'|| {chval(ser["2014→2015"], 2)} ' + \
                  f'|| style="text-align:center;"|{if_value(ser["2015"], 2)} ' + \
                  f'|| {chval(ser["2015→2016"], 2)} ' + \
                  f'|| style="text-align:center;"|{if_value(ser["2016"], 2)} ' + \
                  f'|| {chval(ser["2016→2017"], 2)} ' + \
                  f'|| style="text-align:center;"|{if_value(ser["2017"], 2)} ' + \
                  f'|| {chval(ser["2017→2018"], 2)} ' + \
                  f'|| style="text-align:center;"|{if_value(ser["2018"], 2)} ' + \
                  f'|| {chval(ser["2018→2019"], 2)} ' + \
                  f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["2019"], 2)}\'\'\' ' + \
                  f'|| {chval(ser["2019→2020"], 2)} ' + \
                  f'|| style="text-align:center;"|{if_value(ser["2020"], 2)} ' + \
                  f'|| {chval(ser["2020→2021"], 2)} ' + \
                  f'|| style="text-align:center;"|{if_value(ser["2021"], 2)}'

    if lang == 'ru':
        st = re.sub('(?<=\d)\.(?=\d)', ',', st)  # replace . to comma, if this . is between two digits

    st = table_header + st + '\n' + table_footer
    
    # gray color for missing values
    st = st.replace(';"|—', ';color:silver;"|—')
    
    return st

table_code = create_table_advanced(df_subjects, file_header='rosstat_header_2019_ru.txt', file_footer='rosstat_footer_2019_ru.txt',
                                   dd_replacement=dd_replacement_ru, lang='ru')

In [25]:
# write the code to file
with open('output/Table code for Russian regions -2019 -ru.txt', 'w', encoding="utf-8") as fh:
    fh.write(table_code)

<br />
<br />
<br />

In [26]:
dd_replacement_en = {
    'Россия в среднем': 'Russia on average',
    'Республика Ингушетия': '[[Ingushetia]]',
    'Республика Дагестан': '[[Dagestan]]',
    'г. Москва': '[[Moscow|Moscow]]',
    'Кабардино-Балкарская Республика': '[[Kabardino-Balkaria]]',
    'Карачаево-Черкесская Республика': '[[Karachay-Cherkessia]]',
    'Чеченская Республика': '[[Chechnya]]',
    'г. Санкт-Петербург': '[[Saint Petersburg]]',
    'Республика Северная Осетия-Алания': '[[North Ossetia]]',
    'г. Севастополь': '[[Sevastopol|Sevastopol]]',
    'Ханты-Мансийский АО - Югра (Тюменская область)': '[[Khanty-Mansi Autonomous Okrug|Khanty-Mansi AO]]<br>(Tyumen Oblast)',
    'Ямало-Ненецкий АО (Тюменская область)': '[[Yamalo-Nenets Autonomous Okrug|Yamalo-Nenets AO]]<br>(Tyumen Oblast)',
    'Ставропольский край': '[[Stavropol Krai]]',
    'Республика Калмыкия': '[[Kalmykia]]',
    'Республика Татарстан': '[[Tatarstan]]',
    'Республика Адыгея': '[[Adygea]]',
    'Калининградская область': '[[Kaliningrad Oblast]]',
    'Белгородская область': '[[Belgorod Oblast]]',
    'Краснодарский край': '[[Krasnodar Krai]]',
    'Московская область': '[[Moscow Oblast]]',
    'Республика Мордовия': '[[Mordovia]]',
    'Ленинградская область': '[[Leningrad Oblast]]',
    'Тюменская область (кроме двух АО)': '[[Tyumen Oblast]]<br>(except two AO)',
    'Чувашская Республика (Чувашия)': '[[Chuvashia]]',
    'Удмуртская Республика': '[[Udmurtia]]',
    'Республика Саха (Якутия)': '[[Sakha|Sakha (Yakutia)]]',
    'Пензенская область': '[[Penza Oblast]]',
    'Волгоградская область': '[[Volgograd Oblast]]',
    'Астраханская область': '[[Astrakhan Oblast]]',
    'Тамбовская область': '[[Tambov Oblast]]',
    'Ростовская область': '[[Rostov Oblast]]',
    'Кировская область': '[[Kirov Oblast]]',
    'Республика Крым': '[[Republic of Crimea]]',
    'Томская область': '[[Tomsk Oblast]]',
    'Архангельская область (кроме Ненецкого АО)': '[[Arkhangelsk Oblast]]<br>(except AO)',
    'Воронежская область': '[[Voronezh Oblast]]',
    'Республика Башкортостан': '[[Bashkortostan]]',
    'Республика Марий Эл': '[[Mari El]]',
    'Ненецкий АО (Архангельская область)': '[[Nenets Autonomous Okrug|Nenets AO]]<br>(Arkhangelsk Oblast)',
    'Самарская область': '[[Samara Oblast]]',
    'Новосибирская область': '[[Novosibirsk Oblast]]',
    'Калужская область': '[[Kaluga Oblast]]',
    'Челябинская область': '[[Chelyabinsk Oblast]]',
    'Саратовская область': '[[Saratov Oblast]]',
    'Вологодская область': '[[Vologda Oblast]]',
    'Ярославская область': '[[Yaroslavl Oblast]]',
    'Ульяновская область': '[[Ulyanovsk Oblast]]',
    'Ивановская область': '[[Ivanovo Oblast]]',
    'Омская область': '[[Omsk Oblast]]',
    'Тульская область': '[[Tula Oblast]]',
    'Орловская область': '[[Oryol Oblast]]',
    'Нижегородская область': '[[Nizhny Novgorod Oblast]]',
    'Республика Бурятия': '[[Buryatia]]',
    'Свердловская область': '[[Sverdlovsk Oblast]]',
    'Костромская область': '[[Kostroma Oblast]]',
    'Брянская область': '[[Bryansk Oblast]]',
    'Рязанская область': '[[Ryazan Oblast]]',
    'Приморский край': '[[Primorsky Krai]]',
    'Алтайский край': '[[Altai Krai]]',
    'Липецкая область': '[[Lipetsk Oblast]]',
    'Курская область': '[[Kursk Oblast]]',
    'Пермский край': '[[Perm Krai]]',
    'Республика Хакасия': '[[Khakassia]]',
    'Сахалинская область': '[[Sakhalin Oblast]]',
    'Красноярский край': '[[Krasnoyarsk Krai]]',
    'Республика Коми': '[[Komi Republic|Komi]]',
    'Курганская область': '[[Kurgan Oblast]]',
    'Мурманская область': '[[Murmansk Oblast]]',
    'Оренбургская область': '[[Orenburg Oblast]]',
    'Владимирская область': '[[Vladimir Oblast]]',
    'Камчатский край': '[[Kamchatka Krai]]',
    'Смоленская область': '[[Smolensk Oblast]]',
    'Тверская область': '[[Tver Oblast]]',
    'Республика Алтай': '[[Altai Republic|Altai]]',
    'Хабаровский край': '[[Khabarovsk Krai]]',
    'Псковская область': '[[Pskov Oblast]]',
    'Новгородская область': '[[Novgorod Oblast]]',
    'Кемеровская область - Кузбасс': '[[Kemerovo Oblast|Kuzbass]]',
    'Магаданская область': '[[Magadan Oblast]]',
    'Республика Карелия': '[[Republic of Karelia|Karelia]]',
    'Республика Тыва': '[[Tuva]]',
    'Забайкальский край': '[[Zabaykalsky Krai]]',
    'Иркутская область': '[[Irkutsk Oblast]]',
    'Амурская область': '[[Amur Oblast]]',
    'Еврейская автономная область': '[[Jewish Autonomous Oblast]]',
    'Чукотский АО': '[[Chukotka Autonomous Okrug|Chukotka]]'
}

### version with long names
# dd_replacement_en = {
#     'Россия в среднем': 'Russia on average',
#     'Республика Ингушетия': '[[Ingushetia|Republic of Ingushetia]]',
#     'Республика Дагестан': '[[Dagestan|Republic of Dagestan]]',
#     'г. Москва': '[[Moscow|federal city Moscow]]',
#     'Кабардино-Балкарская Республика': '[[Kabardino-Balkaria|Kabardino-Balkarian Republic]]',
#     'Карачаево-Черкесская Республика': '[[Karachay-Cherkessia|Karachay-Cherkess Republic]]',
#     'Чеченская Республика': '[[Chechnya|Chechen Republic]]',
#     'г. Санкт-Петербург': '[[Saint Petersburg|federal city Saint Petersburg]]',
#     'Республика Северная Осетия-Алания': '[[North Ossetia–Alania|Republic of North Ossetia–Alania]]',
#     'г. Севастополь': '[[Sevastopol|federal city Sevastopol]]',
#     'Ханты-Мансийский АО - Югра (Тюменская область)': '[[Khanty-Mansi Autonomous Okrug|Khanty-Mansi AO–Yugra]]<br>(Tyumen Oblast)',
#     'Ямало-Ненецкий АО (Тюменская область)': '[[Yamalo-Nenets Autonomous Okrug|Yamalo-Nenets AO]]<br>(Tyumen Oblast)',
#     'Ставропольский край': '[[Stavropol Krai]]',
#     'Республика Калмыкия': '[[Kalmykia|Republic of Kalmykia]]',
#     'Республика Татарстан': '[[Tatarstan|Republic of Tatarstan]]',
#     'Республика Адыгея': '[[Adygea|Republic of Adygea]]',
#     'Калининградская область': '[[Kaliningrad Oblast]]',
#     'Белгородская область': '[[Belgorod Oblast]]',
#     'Краснодарский край': '[[Krasnodar Krai]]',
#     'Московская область': '[[Moscow Oblast]]',
#     'Республика Мордовия': '[[Mordovia|Republic of Mordovia]]',
#     'Ленинградская область': '[[Leningrad Oblast]]',
#     'Тюменская область (кроме двух АО)': '[[Tyumen Oblast]]<br>(except two AO)',
#     'Чувашская Республика (Чувашия)': '[[Chuvashia|Chuvash Republic]]',
#     'Удмуртская Республика': '[[Udmurtia|Udmurt Republic]]',
#     'Республика Саха (Якутия)': '[[Sakha|Republic of Sakha (Yakutia)]]',
#     'Пензенская область': '[[Penza Oblast]]',
#     'Волгоградская область': '[[Volgograd Oblast]]',
#     'Астраханская область': '[[Astrakhan Oblast]]',
#     'Тамбовская область': '[[Tambov Oblast]]',
#     'Ростовская область': '[[Rostov Oblast]]',
#     'Кировская область': '[[Kirov Oblast]]',
#     'Республика Крым': '[[Republic of Crimea]]',
#     'Томская область': '[[Tomsk Oblast]]',
#     'Архангельская область (кроме Ненецкого АО)': '[[Arkhangelsk Oblast]]<br>(except Nenets AO])',
#     'Воронежская область': '[[Voronezh Oblast]]',
#     'Республика Башкортостан': '[[Bashkortostan|Republic of Bashkortostan]]',
#     'Республика Марий Эл': '[[Mari El|Mari El Republic]]',
#     'Ненецкий АО (Архангельская область)': '[[Nenets Autonomous Okrug|Nenets AO]]<br>(Arkhangelsk Oblast)',
#     'Самарская область': '[[Samara Oblast]]',
#     'Новосибирская область': '[[Novosibirsk Oblast]]',
#     'Калужская область': '[[Kaluga Oblast]]',
#     'Челябинская область': '[[Chelyabinsk Oblast]]',
#     'Саратовская область': '[[Saratov Oblast]]',
#     'Вологодская область': '[[Vologda Oblast]]',
#     'Ярославская область': '[[Yaroslavl Oblast]]',
#     'Ульяновская область': '[[Ulyanovsk Oblast]]',
#     'Ивановская область': '[[Ivanovo Oblast]]',
#     'Омская область': '[[Omsk Oblast]]',
#     'Тульская область': '[[Tula Oblast]]',
#     'Орловская область': '[[Oryol Oblast]]',
#     'Нижегородская область': '[[Nizhny Novgorod Oblast]]',
#     'Республика Бурятия': '[[Buryatia|Republic of Buryatia]]',
#     'Свердловская область': '[[Sverdlovsk Oblast]]',
#     'Костромская область': '[[Kostroma Oblast]]',
#     'Брянская область': '[[Bryansk Oblast]]',
#     'Рязанская область': '[[Ryazan Oblast]]',
#     'Приморский край': '[[Primorsky Krai]]',
#     'Алтайский край': '[[Altai Krai]]',
#     'Липецкая область': '[[Lipetsk Oblast]]',
#     'Курская область': '[[Kursk Oblast]]',
#     'Пермский край': '[[Perm Krai]]',
#     'Республика Хакасия': '[[Khakassia|Republic of Khakassia]]',
#     'Сахалинская область': '[[Sakhalin Oblast]]',
#     'Красноярский край': '[[Krasnoyarsk Krai]]',
#     'Республика Коми': '[[Komi Republic]]',
#     'Курганская область': '[[Kurgan Oblast]]',
#     'Мурманская область': '[[Murmansk Oblast]]',
#     'Оренбургская область': '[[Orenburg Oblast]]',
#     'Владимирская область': '[[Vladimir Oblast]]',
#     'Камчатский край': '[[Kamchatka Krai]]',
#     'Смоленская область': '[[Smolensk Oblast]]',
#     'Тверская область': '[[Tver Oblast]]',
#     'Республика Алтай': '[[Altai Republic]]',
#     'Хабаровский край': '[[Khabarovsk Krai]]',
#     'Псковская область': '[[Pskov Oblast]]',
#     'Новгородская область': '[[Novgorod Oblast]]',
#     'Кемеровская область - Кузбасс': '[[Kemerovo Oblast|Kemerovo Oblast–Kuzbass]]',
#     'Магаданская область': '[[Magadan Oblast]]',
#     'Республика Карелия': '[[Republic of Karelia]]',
#     'Республика Тыва': '[[Tuva|Tyva Republic]]',
#     'Забайкальский край': '[[Zabaykalsky Krai]]',
#     'Иркутская область': '[[Irkutsk Oblast]]',
#     'Амурская область': '[[Amur Oblast]]',
#     'Еврейская автономная область': '[[Jewish Autonomous Oblast]]',
#     'Чукотский АО': '[[Chukotka Autonomous Okrug|Chukotka AO]]'
# }

In [27]:
# table_code = create_table(df_subjects, file_header='rosstat_header_2019_en.txt', file_footer='rosstat_footer_2019_en.txt', dd_replacement=dd_replacement_en)

# # write the code to file
# with open('output/Table code for Russian regions -2019 -en.txt', 'w', encoding="utf-8") as fh:
#     fh.write(table_code)

In [28]:
table_code = create_table_advanced(df_subjects, file_header='rosstat_header_2019_en.txt', file_footer='rosstat_footer_2019_en.txt',
                                   dd_replacement=dd_replacement_en, lang='en')

In [29]:
# write the code to file
with open('output/Table code for Russian regions -2019 -en.txt', 'w', encoding="utf-8") as fh:
    fh.write(table_code)

<br />
<br />
<br />

#### Auxiliary table that I decided not to use

In [30]:
ls_attention = [
    'Архангельская область',
    'Архангельская область (кроме Ненецкого АО)',
    'Ненецкий АО (Архангельская область)',
    'Тюменская область',
    'Тюменская область (кроме двух АО)', 
    'Ханты-Мансийский АО - Югра (Тюменская область)',
    'Ямало-Ненецкий АО (Тюменская область)',
]

df_auxiliary = df.loc[ls_attention]
print(df_auxiliary.shape)
df_auxiliary.fillna(' ')

(7, 22)


,overall,males,females,females_vs_males,urban,rural,urban_vs_rural,2014,2014→2015,2015,2015→2016,2016,2016→2017,2017,2017→2018,2018,2018→2019,2019,2019→2020,2020,2020→2021,2021
Архангельская область,72.34,66.81,77.72,10.91,73.40,67.95,5.45,70.23,0.48,70.71,0.11,70.82,1.12,71.94,0.16,72.10,0.24,72.34,-0.99,71.35,-1.75,69.60
Архангельская область (кроме Ненецкого АО),72.30,66.77,77.68,10.91,73.33,67.94,5.39,70.20,0.50,70.70,0.10,70.80,1.16,71.96,0.13,72.09,0.21,72.30,-0.91,71.39,-1.79,69.60
Ненецкий АО (Архангельская область),73.19,67.68,78.40,10.72,75.70,67.41,8.29,70.65,0.35,71.00,0.08,71.08,0.44,71.52,0.33,71.85,1.34,73.19,-2.79,70.40,-1.01,69.39
Тюменская область,74.04,69.17,78.69,9.52,74.89,70.80,4.09,71.50,0.26,71.76,0.57,72.33,0.77,73.10,0.30,73.40,0.64,74.04,-1.80,72.24,-0.98,71.26
Тюменская область (кроме двух АО),72.75,67.52,77.83,10.31,73.93,70.54,3.39,70.32,0.26,70.58,0.45,71.03,1.03,72.06,0.02,72.08,0.67,72.75,-1.41,71.34,-1.20,70.14
Ханты-Мансийский АО - Югра (Тюменская область),75.04,70.60,79.28,8.68,75.24,72.59,2.65,72.27,0.31,72.58,0.92,73.50,0.37,73.87,0.41,74.28,0.76,75.04,-2.17,72.87,-0.86,72.01
Ямало-Ненецкий АО (Тюменская область),74.18,69.39,78.61,9.22,74.98,69.97,5.01,71.92,-0.22,71.70,0.43,72.13,1.40,73.53,0.54,74.07,0.11,74.18,-2.27,71.91,-0.21,71.70


In [31]:
# create code for placing info in Wikipedia
# def create_table_auxiliary(df, file_header):

#     def if_value(x, prec=2):
#         return '-' if math.isnan(x) else f"{x:0.{prec}f}" #"{x:0.{prec}f}".format(x, prec)
    
#     with open('design/' + file_header, mode='r', encoding="utf-8") as fh:
#         table_header = fh.read()

#     # st = table_header
#     st = table_header
#     for i in range(len(df)):
#         ser = df.iloc[i]       
#         st += '\n' + '|-\n' + \
#               f'|align=left|{dd_replacement_ru[ser.name]} ' + \
#               f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["overall"], 2)}\'\'\' ' + \
#               f'|| style="background:#eaf3ff;"|{if_value(ser["males"], 2)} ' + \
#               f'|| style="background:#fee7f6;"|{if_value(ser["females"], 2)} ' + \
#               f'|| style="background:#fff8dc;"|{if_value(ser["females_vs_males"], 2)} ' + \
#               f'|| style="border-left-width:2px;"|{if_value(ser["urban"], 2)} ' + \
#               f'|| {if_value(ser["rural"], 2)} ' + \
#               f'|| style="background:#fff8dc;"|{if_value(ser["urban_vs_rural"], 2)} ' + \
#               f'|| style="border-left-width:2px;"|{if_value(ser["1991"], 1)} ' + \
#               f'|| style="background:#fff8dc;"|{if_value(ser["1991→2000"], 1)} ' + \
#               f'|| {if_value(ser["2000"], 1)} ' + \
#               f'|| style="background:#fff8dc;"|{if_value(ser["2000→2014"], 2)} ' + \
#               f'|| {if_value(ser["2014"], 2)} ' + \
#               f'|| style="background:#fff8dc;"|{if_value(ser["2014→2019"], 2)} ' + \
#               f'|| {if_value(ser["2019"], 2)} ' + \
#               f'|| style="background:#fff8dc;"|{if_value(ser["2019→2020"], 2)} ' + \
#               f'|| {if_value(ser["2020"], 2)} ' + \
#               f'|| style="background:#fff8dc;"|{if_value(ser["2020→2021"], 2)} ' + \
#               f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["2021"], 2)}\'\'\''
#     st += '\n' + '}}'
#     return st

# table_code = create_table_auxiliary(df_auxiliary, file_header='rosstat_header_ru.txt')
# print(table_code)

In [32]:
# write the code to file
# with open('output/Table code for Russian regions -attention.txt', 'w', encoding="utf-8") as fh:
#     fh.write(table_code)

# Naming of some regions are reqiered small manual asjustment for proper linking to Wikipedia pages of according redions.
# Ideally this should be make via dictionary - this code is not professional.